In [1]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [2]:
using Revise
using LinearAlgebra
using Distributions
using PDMats
using StatsBase
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [3]:
include("../VARs/utils.jl")
includet("../bootstrap_model.jl")

In [4]:
df = DataFrame(XLSX.readtable("../../data/usa_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

freq = 12
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))


display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1473,1.8543,0.1012,0.4365,-2.9008,-0.6469,-0.1883,0.0258,2.1941,0.6874,0.1474
Price_10Y,0.0755,1.3969,0.0531,0.1909,-0.9866,0.3097,-0.0893,0.1022,1.9867,0.7037,0.0377


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1479,1.7607,0.0658,0.4053,-2.9112,-0.596,-0.2186,0.032,1.9119,0.445,0.1107
Price_10Y_CPI,0.078,1.279,0.0177,0.1544,-1.1677,0.2268,-0.1322,0.1437,2.0153,0.2268,0.0058


nothing

"Correlations"

2×2 Matrix{Float64}:
 1.0        0.0980477
 0.0980477  1.0

In [18]:
assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
data = to(from(real_returns[assets_names], start),end_d) 

scenarios = block_bootstrap(values(data), 10_000, block_size=36, random_block=true)

periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  


,Total_Stock_CPI,Price_10Y_CPI
1,0.0659,0.0232
5,0.0633,0.0205
10,0.0629,0.0205
25,0.0628,0.0202


,Total_Stock_CPI,Price_10Y_CPI
1,0.1581,0.0877
5,0.0674,0.0435
10,0.048,0.0315
25,0.0302,0.0201


,Total_Stock_CPI,Price_10Y_CPI
1,-0.7649,0.3988
5,-0.3457,0.1075
10,-0.2711,0.092
25,-0.1843,0.0753


,Total_Stock_CPI,Price_10Y_CPI
1,1.1648,0.8119
5,0.0726,0.6231
10,-0.0866,0.0783
25,-0.0728,-0.0468


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.4418,-0.2887,-0.2145,-0.0207,0.0845,0.1744,0.2895,0.3263,0.3825
5,-0.1053,-0.0741,-0.0513,0.0174,0.0714,0.1104,0.1653,0.1856,0.2061
10,-0.0558,-0.0355,-0.0205,0.0307,0.0664,0.097,0.1362,0.1483,0.163
25,-0.0097,0.0026,0.0116,0.0426,0.064,0.0844,0.1101,0.1187,0.1282


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1745,-0.1322,-0.1103,-0.0345,0.0196,0.0785,0.1716,0.2258,0.285
5,-0.0828,-0.0694,-0.0541,-0.0054,0.0199,0.0473,0.0925,0.1106,0.1402
10,-0.0513,-0.0407,-0.0312,-0.0006,0.0204,0.041,0.0735,0.0847,0.096
25,-0.0252,-0.0184,-0.012,0.0066,0.0199,0.0338,0.0536,0.0603,0.0673


In [17]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.354481  0.482982  0.536639  0.609397  0.759716

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 55.0  95.0  131.0  187.25  370.0

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.239173  0.339566  0.407344  0.491454  0.666609

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 96.0  171.75  251.0  373.0  695.0

In [7]:
rr = values(to(from(real_returns[assets_names], start),end_d) ) .- mean(values(to(from(real_returns[assets_names], start),end_d) ), dims=1)


scenarios = block_bootstrap(rr, 10_000, block_size=60, random_block=true)


scenarios[1,:,:] =  scenarios[1,:,:] .+ .046 ./ 12


scenarios[2,:,:] =  scenarios[2,:,:] .+ .02 ./ 12

835×10000 Matrix{Float64}:
  0.025504      0.0270657     0.0121892    …  -0.00785432  -0.0490227
  0.0201573    -0.00637662    0.000383427     -0.00834786   0.0349321
 -0.0021086     0.0150916    -0.0215482       -0.00838018  -0.0186625
  0.00674751   -0.00158563   -0.000454571     -0.0156902   -0.0379845
  0.0450771     0.0598528     0.00961533      -0.0136035   -0.0346014
 -0.00713332    0.00486736   -0.0107024    …   0.00671513  -0.0103039
  0.0306224     0.0326678    -0.0129857        0.0109046    0.0697859
  0.00614153   -0.000909451  -0.0194724       -0.0130171    0.0868955
  0.00371241    0.0476314     0.00547431      -0.0232386   -0.0375228
 -0.031869     -0.0413481    -0.00788294       0.00963055   0.000264783
  ⋮                                        ⋱               
 -0.0119338    -0.0127218    -0.00710394       0.0448111   -0.0237272
 -0.00317821    0.00974712   -0.0106819       -0.0122648    0.00408265
  0.0087302    -0.01974      -0.0259628       -0.0787728    0.00450673

In [8]:
periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)

print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  


,Total_Stock_CPI,Price_10Y_CPI
1,0.0414,0.0267
5,0.0426,0.0253
10,0.0425,0.0247
25,0.0427,0.0245


,Total_Stock_CPI,Price_10Y_CPI
1,0.1554,0.0861
5,0.068,0.0419
10,0.0483,0.0309
25,0.0302,0.0206


,Total_Stock_CPI,Price_10Y_CPI
1,-0.8103,0.4879
5,-0.2634,0.2755
10,-0.3342,0.1545
25,-0.183,0.1377


,Total_Stock_CPI,Price_10Y_CPI
1,1.2634,0.9599
5,-0.2196,0.7835
10,-0.3154,-0.0269
25,-0.1764,-0.177


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.4616,-0.3243,-0.2326,-0.042,0.062,0.1432,0.2602,0.2918,0.3417
5,-0.113,-0.0926,-0.0728,-0.0064,0.0515,0.0901,0.1468,0.1699,0.1891
10,-0.0724,-0.0553,-0.0435,0.0098,0.0465,0.0785,0.1153,0.1239,0.1364
25,-0.0288,-0.019,-0.0088,0.0221,0.0441,0.0642,0.0898,0.0978,0.1066


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1595,-0.1219,-0.0984,-0.0318,0.022,0.0809,0.1708,0.2286,0.2875
5,-0.0728,-0.0571,-0.0404,-0.0013,0.0233,0.0514,0.0961,0.1167,0.1437
10,-0.0431,-0.0336,-0.0237,0.003,0.0244,0.0448,0.0788,0.0896,0.0991
25,-0.0196,-0.0136,-0.0086,0.0098,0.0239,0.0386,0.0592,0.0664,0.073


In [9]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.441017  0.537691  0.601903  0.659362  0.792846

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 75.0  150.0  201.0  282.0  546.0

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.193099  0.371434  0.417165  0.481736  0.632742

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 75.0  147.0  211.0  309.0  611.03